In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import geopy
from geopy import Point
from geopy.distance import geodesic
from shapely.geometry import Polygon
import fiona
import time
import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_seq_items', None)

In [3]:
df = pd.read_csv('data/gmonpro_20221207_090716_Sim1_44011.csv', header=None)

In [4]:
df.head(3)

,0
0,PLMN;SYSTEM;XCI;xNBID;LOCAL_CID;LAC/TAC;PCI/PSC/BSIC;ARFCN;BAND;RSSI;RSRP/RSCP;RSRQ/ECIO;SNR;CQI;TA;DISTANCE;DELTA_AZI;LAT;LON;SPEED;GPS_ACCURACY;UL;DL;BANDWIDTH;BANDWIDTHS;CA;NR_STATE;NARFCN;NR_BAND;NR_PCI;NR_SS_RSRP;NR_SS_RSRQ;NR_SS_SINR;NR_CSI_RSRP;NR_CSI_RSRQ;NR_CSI_SINR;CLF_LABEL;CLF_LOC;CLF_DESC;DATE;TIME;ROAMING
1,44011;4;71726856;280183;8;211;251;1575;1800 B3;-65;-92;-6;1.0;;;;;35.698181;139.773952;0;25;22;120;5000;5;1;none;;;;;;;;;;--;--;Cell not found in the clf database!;2022/12/07;09:07:16;HOME
2,44011;4;71726856;280183;8;211;251;1575;1800 B3;-65;-92;-6;1.0;;56;;;35.698181;139.773952;0;25;11;9;5000;5;1;none;;;;;;;;;;--;--;Cell not found in the clf database!;2022/12/07;09:07:17;HOME


In [5]:
df = df[0].str.split(';', expand=True)
df.columns = df.iloc[0]
df.drop(0, inplace=True)
df.reset_index(drop=True, inplace=True)
df.rename(columns={'PCI/PSC/BSIC':'PCI', 'RSRP/RSCP':'RSRP', 'RSRQ/ECIO':'RSRQ', 'SNR':'SINR', 'LAT':'LAT_org', 'LON':'LON_org', 'TIME':'TIME_org'}, inplace=True)
df.insert(0, 'LON', df['LON_org'])
df.insert(0, 'LAT', df['LAT_org'])
df.insert(0, 'TIME', df['TIME_org'])
df.drop(['LON_org', 'LAT_org', 'TIME_org'], axis=1, inplace=True)

In [6]:
df['PCI'].nunique()

21

In [7]:
df.drop_duplicates('PCI', inplace=True)
df.reset_index(drop=True, inplace=True)

In [8]:
df['PCI'].nunique()

21

In [9]:
df.head(3)

,TIME,LAT,LON,PLMN,SYSTEM,XCI,xNBID,LOCAL_CID,LAC/TAC,PCI,ARFCN,BAND,RSSI,RSRP,RSRQ,SINR,CQI,TA,DISTANCE,DELTA_AZI,SPEED,GPS_ACCURACY,UL,DL,BANDWIDTH,BANDWIDTHS,CA,NR_STATE,NARFCN,NR_BAND,NR_PCI,NR_SS_RSRP,NR_SS_RSRQ,NR_SS_SINR,NR_CSI_RSRP,NR_CSI_RSRQ,NR_CSI_SINR,CLF_LABEL,CLF_LOC,CLF_DESC,DATE,ROAMING
0,09:07:16,35.698181,139.773952,44011,4,71726856,280183,8,211,251,1575,1800 B3,-65,-92,-6,1.0,,,,,0,25,22,120,5000,5,1,none,,,,,,,,,,--,--,Cell not found in the clf database!,2022/12/07,HOME
1,09:07:38,35.698235,139.773568,44011,4,71726855,280183,7,211,15,1575,1800 B3,-77,-100,-13,0.8,,,,,10,63,0,0,5000,5,1,none,,,,,,,,,,--,--,Cell not found in the clf database!,2022/12/07,HOME
2,09:07:48,35.698445,139.772364,44011,4,71434506,279041,10,211,123,1500,1800 B3,-63,-87,-12,-0.3,,,,,27,27,5,4,20000,20,1,none,,,,,,,,,,--,--,Cell not found in the clf database!,2022/12/07,HOME


In [10]:
df['AZIMUTH'] = np.random.randint(0, 36, size=21)*10

In [11]:
df.columns

Index(['TIME', 'LAT', 'LON', 'PLMN', 'SYSTEM', 'XCI', 'xNBID', 'LOCAL_CID',
       'LAC/TAC', 'PCI', 'ARFCN', 'BAND', 'RSSI', 'RSRP', 'RSRQ', 'SINR',
       'CQI', 'TA', 'DISTANCE', 'DELTA_AZI', 'SPEED', 'GPS_ACCURACY', 'UL',
       'DL', 'BANDWIDTH', 'BANDWIDTHS', 'CA', 'NR_STATE', 'NARFCN', 'NR_BAND',
       'NR_PCI', 'NR_SS_RSRP', 'NR_SS_RSRQ', 'NR_SS_SINR', 'NR_CSI_RSRP',
       'NR_CSI_RSRQ', 'NR_CSI_SINR', 'CLF_LABEL', 'CLF_LOC', 'CLF_DESC',
       'DATE', 'ROAMING', 'AZIMUTH'],
      dtype='object', name=0)

In [12]:
df = df[['LAT', 'LON', 'AZIMUTH', 'XCI', 'xNBID', 'LOCAL_CID', 'LAC/TAC', 'PCI', 'ARFCN', 'BAND', 'BANDWIDTH']]

In [13]:
df[['LAT', 'LON']] = df[['LAT', 'LON']].astype(np.float64)

In [14]:
df.head()

,LAT,LON,AZIMUTH,XCI,xNBID,LOCAL_CID,LAC/TAC,PCI,ARFCN,BAND,BANDWIDTH
0,35.698181,139.773952,80,71726856,280183,8,211,251,1575,1800 B3,5000
1,35.698235,139.773568,70,71726855,280183,7,211,15,1575,1800 B3,5000
2,35.698445,139.772364,50,71434506,279041,10,211,123,1500,1800 B3,20000
3,35.698445,139.772364,270,71434507,279041,11,211,95,1500,1800 B3,20000
4,35.698210,139.768710,230,71403531,278920,11,211,365,1500,1800 B3,20000


In [15]:
for i, lat, lon, azi in zip(df.index, df['LAT'], df['LON'], df['AZIMUTH']):
    df.loc[i, 'A Point'] = geodesic(meters=1000/np.cos(np.radians(30))).destination(Point(lat, lon), azi+30).format_decimal()
    df.loc[i, 'B Point'] = geodesic(meters=1000/np.cos(np.radians(30))).destination(Point(lat, lon), azi-30).format_decimal()

In [16]:
df.head()

,LAT,LON,AZIMUTH,XCI,xNBID,LOCAL_CID,LAC/TAC,PCI,ARFCN,BAND,BANDWIDTH,A Point,B Point
0,35.698181,139.773952,80,71726856,280183,8,211,251,1575,1800 B3,5000,"35.69462097261708, 139.7859403258893","35.70487013844407, 139.783726223506"
1,35.698235,139.773568,70,71726855,280183,7,211,15,1575,1800 B3,5000,"35.69642717439867, 139.7861321749687","35.706206997354, 139.78176968425632"
2,35.698445,139.772364,50,71434506,279041,10,211,123,1500,1800 B3,20000,"35.70025151353197, 139.78492877489285","35.70822436425991, 139.77672813498566"
3,35.698445,139.772364,270,71434507,279041,11,211,95,1500,1800 B3,20000,"35.7036480279664, 139.76131425378574","35.693240953343846, 139.76131568944058"
4,35.698210,139.768710,230,71403531,278920,11,211,365,1500,1800 B3,20000,"35.69640217439174, 139.7561458289527","35.68843046125399, 139.76434694339318"


In [17]:
a_set = df['A Point'].str.split(',', expand=True)
df['A Lat'] = a_set[0].astype(np.float64)
df['A Lon'] = a_set[1].astype(np.float64)

b_set = df['B Point'].str.split(',', expand=True)
df['B Lat'] = b_set[0].astype(np.float64)
df['B Lon'] = b_set[1].astype(np.float64)

In [18]:
df.head()

,LAT,LON,AZIMUTH,XCI,xNBID,LOCAL_CID,LAC/TAC,PCI,ARFCN,BAND,BANDWIDTH,A Point,B Point,A Lat,A Lon,B Lat,B Lon
0,35.698181,139.773952,80,71726856,280183,8,211,251,1575,1800 B3,5000,"35.69462097261708, 139.7859403258893","35.70487013844407, 139.783726223506",35.694621,139.785940,35.704870,139.783726
1,35.698235,139.773568,70,71726855,280183,7,211,15,1575,1800 B3,5000,"35.69642717439867, 139.7861321749687","35.706206997354, 139.78176968425632",35.696427,139.786132,35.706207,139.781770
2,35.698445,139.772364,50,71434506,279041,10,211,123,1500,1800 B3,20000,"35.70025151353197, 139.78492877489285","35.70822436425991, 139.77672813498566",35.700252,139.784929,35.708224,139.776728
3,35.698445,139.772364,270,71434507,279041,11,211,95,1500,1800 B3,20000,"35.7036480279664, 139.76131425378574","35.693240953343846, 139.76131568944058",35.703648,139.761314,35.693241,139.761316
4,35.698210,139.768710,230,71403531,278920,11,211,365,1500,1800 B3,20000,"35.69640217439174, 139.7561458289527","35.68843046125399, 139.76434694339318",35.696402,139.756146,35.688430,139.764347


In [19]:
df['S Coord'] = list(zip(df['LON'], df['LAT']))
df['A Coord'] = list(zip(df['A Lon'], df['A Lat']))
df['B Coord'] = list(zip(df['B Lon'], df['B Lat']))

df['Coordinates'] = list(zip(df['S Coord'], df['A Coord'], df['B Coord']))

In [20]:
df['Coord_Polygon'] = df['Coordinates'].apply(Polygon)

In [21]:
df['Coord_Polygon'].head()

0    POLYGON ((139.773952 35.698181, 139.7859403258...
1    POLYGON ((139.773568 35.698235, 139.7861321749...
2    POLYGON ((139.772364 35.698445, 139.7849287748...
3    POLYGON ((139.772364 35.698445, 139.7613142537...
4    POLYGON ((139.76871 35.69821, 139.756145828952...
Name: Coord_Polygon, dtype: object

In [22]:
gdf = gpd.GeoDataFrame(df, geometry = df['Coord_Polygon'])

In [23]:
gdf.columns

Index(['LAT', 'LON', 'AZIMUTH', 'XCI', 'xNBID', 'LOCAL_CID', 'LAC/TAC', 'PCI',
       'ARFCN', 'BAND', 'BANDWIDTH', 'A Point', 'B Point', 'A Lat', 'A Lon',
       'B Lat', 'B Lon', 'S Coord', 'A Coord', 'B Coord', 'Coordinates',
       'Coord_Polygon', 'geometry'],
      dtype='object', name=0)

In [24]:
gdf.drop(['A Point', 'B Point', 'S Coord', 'A Coord', 'B Coord', 'Coordinates',
       'Coord_Polygon'], axis=1, inplace=True)

In [25]:
gdf.columns

Index(['LAT', 'LON', 'AZIMUTH', 'XCI', 'xNBID', 'LOCAL_CID', 'LAC/TAC', 'PCI',
       'ARFCN', 'BAND', 'BANDWIDTH', 'A Lat', 'A Lon', 'B Lat', 'B Lon',
       'geometry'],
      dtype='object', name=0)

In [26]:
gdf = gdf[['XCI', 'xNBID', 'LOCAL_CID', 'LAC/TAC', 'PCI', 'ARFCN', 'BAND', 'BANDWIDTH', 'AZIMUTH', 
           'LAT', 'LON', 'A Lat', 'A Lon', 'B Lat', 'B Lon', 'geometry']]

In [27]:
fiona.supported_drivers['KML'] = 'rw'
fiona.drvsupport.supported_drivers['KML'] = 'rw'
fiona.drvsupport.supported_drivers['libkml'] = 'rw' # enable KML support which is disabled by default
fiona.drvsupport.supported_drivers['LIBKML'] = 'rw' # enable KML support which is disabled 

In [28]:
# Checks if GeoPandas can read MapInfo files
print('MapInfo File' in fiona.supported_drivers)

True


In [29]:
#gdf.to_file('../a.kml', driver='KML')